In [ ]:
import torch, subprocess, os, textwrap
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
!nvidia-smi -L  # should list a GPU if you got one

!pip install -q seqeval

import itertools, pandas as pd

def read_conll(path):
    """Reads CoNLL file -> list[ list[str] ], list[ list[str] ]"""
    sents, labels, tokens, tags = [], [], [], []
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("-DOCSTART-"):
                if tokens:
                    sents.append(tokens); labels.append(tags)
                    tokens, tags = [], []
                continue
            token, tag = line.split()[0], line.split()[-1]
            tokens.append(token); tags.append(tag)
    if tokens:                      # last sentence
        sents.append(tokens); labels.append(tags)
    return sents, labels

train_s, train_l = read_conll("/content/train.txt")
val_s,   val_l   = read_conll("/content/valid.txt")
test_s,  test_l  = read_conll("/content/test.txt")

print("Train:", len(train_s), "Val:", len(val_s), "Test:", len(test_s))



unique_tags = sorted({t for seq in itertools.chain(train_l, val_l, test_l) for t in seq})
id2tag      = {i:t for i,t in enumerate(unique_tags)}
tag2id      = {t:i for i,t in id2tag.items()}
num_labels  = len(unique_tags)
print("Labels:", unique_tags)


from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def encode(sentences, labels):
    encodings = tokenizer(
        sentences,                  # list[list[str]]
        is_split_into_words=True,
        return_offsets_mapping=True,
        truncation=True,
        padding=True
    )
    encoded_labels = []
    for i, offsets in enumerate(encodings.pop("offset_mapping")):
        word_ids = encodings.word_ids(batch_index=i)
        label_ids = []
        prev_word = None
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != prev_word:
                label_ids.append(tag2id[ labels[i][word_id] ])
                prev_word = word_id
            else:                       # sub-token of same word
                label_ids.append(-100)
        encoded_labels.append(label_ids)
    encodings["labels"] = encoded_labels
    return encodings

train_enc = encode(train_s, train_l)
val_enc   = encode(val_s,   val_l)
test_enc  = encode(test_s,  test_l)


import torch
class NERDataset(torch.utils.data.Dataset):
    def __init__(self, enc): self.enc = enc
    def __len__(self):       return len(self.enc["input_ids"])
    def __getitem__(self, i):
        return {k: torch.tensor(v[i]) for k,v in self.enc.items()}

train_ds, val_ds, test_ds = map(NERDataset, [train_enc, val_enc, test_enc])


from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=num_labels,
    id2label=id2tag,
    label2id=tag2id
)


from transformers import TrainingArguments, Trainer
import numpy as np
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

def metrics_fn(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    true_tags, pred_tags = [], []
    for p, l in zip(preds, labels):
        seq_true, seq_pred = [], []
        for pi, li in zip(p, l):
            if li == -100:             # ignore sub-tokens / padding
                continue
            seq_true.append(id2tag[li])
            seq_pred.append(id2tag[pi])
        true_tags.append(seq_true)
        pred_tags.append(seq_pred)
    return {
        "precision": precision_score(true_tags, pred_tags),
        "recall":    recall_score(true_tags, pred_tags),
        "f1":        f1_score(true_tags, pred_tags)
    }

args = TrainingArguments(
    output_dir          = "ner_modelV3",
    eval_strategy       = "steps",
    save_strategy       = "steps",
    learning_rate       = 6e-6,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 16,
    num_train_epochs    = 5,
    weight_decay        = 0.01,
    logging_steps       = 100,
    load_best_model_at_end = True,
    metric_for_best_model = "f1"
)

trainer = Trainer(
    model           = model,
    args            = args,
    train_dataset   = train_ds,
    eval_dataset    = val_ds,
    compute_metrics = metrics_fn,
    tokenizer       = tokenizer
)


trainer.train()


metrics = trainer.evaluate(test_ds)
print(metrics)


pred_logits = trainer.predict(test_ds).predictions
pred_ids    = np.argmax(pred_logits, axis=-1)

true_tags, pred_tags = [], []
for p, l in zip(pred_ids, test_enc["labels"]):
    t_seq, p_seq = [], []
    for pi, li in zip(p, l):
        if li == -100: continue
        t_seq.append(id2tag[li]); p_seq.append(id2tag[pi])
    true_tags.append(t_seq); pred_tags.append(p_seq)

print(classification_report(true_tags, pred_tags))


trainer.save_model("/content/ner_bert_NLPv3")
tokenizer.save_pretrained("/content/ner_bert_NLPv3")


import shutil, pathlib, zipfile

ckpt_dir = pathlib.Path("/content/ner_bert_NLPv3")
zip_path = "/content/ner_bert_NLPv3.zip"

shutil.make_archive(base_name=zip_path.replace(".zip",""),
                    format="zip",
                    root_dir=ckpt_dir)

from google.colab import files
files.download(zip_path)        # Tarayıcınıza iner


from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch, numpy as np

model_dir = "/content/ner_bert_NLPv3"   # ya da indirdiğiniz zip’i açtığınız yer
tok  = AutoTokenizer.from_pretrained(model_dir)
ner  = AutoModelForTokenClassification.from_pretrained(model_dir)

sentence = "Donald Trump and Donald Duck is an amazing person who works at Facebook and Google at California"
tokens   = tok(sentence, return_tensors="pt")
with torch.no_grad():
    logits = ner(**tokens).logits
pred_ids = logits.argmax(-1).squeeze().tolist()

for sub_tok, tag_id in zip(tok.tokenize(sentence), pred_ids[1:-1]):  # CLS-SEP hariç
    tag = ner.config.id2label[tag_id]
    print(f"{sub_tok:12} → {tag}")


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 1) Numeric Results
val_metrics = trainer.evaluate(val_ds)
test_metrics = trainer.evaluate(test_ds)

print("Validation Metrics:")
for k, v in val_metrics.items():
    if k.startswith("eval_"):
        print(f"  {k}: {v:.4f}")

print("\nTest Metrics:")
for k, v in test_metrics.items():
    if k.startswith("eval_"):
        print(f"  {k}: {v:.4f}")

# 2) Visual Results from log_history
history = pd.DataFrame(trainer.state.log_history)

# a) Train & Eval loss vs step
train_loss_df = history[['step', 'loss']].dropna()
eval_df       = history[['step', 'eval_loss', 'eval_precision', 'eval_recall', 'eval_f1']].dropna()

plt.figure(figsize=(10, 6), dpi=200)
plt.plot(train_loss_df['step'], train_loss_df['loss'], label='train_loss', linewidth=1.5)
plt.plot(eval_df['step'],       eval_df['eval_loss'], label='eval_loss',  linewidth=1.5)
plt.xlabel('Step')
plt.ylabel('Loss')
plt.title('Training & Evaluation Loss vs. Step')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# b) Precision / Recall / F1 vs step
plt.figure(figsize=(10, 6), dpi=200)
plt.plot(eval_df['step'], eval_df['eval_precision'], marker='o', label='Precision')
plt.plot(eval_df['step'], eval_df['eval_recall'],    marker='o', label='Recall')
plt.plot(eval_df['step'], eval_df['eval_f1'],        marker='o', label='F1-score')
plt.xlabel('Step')
plt.ylabel('Score')
plt.title('Evaluation Metrics vs. Step')
plt.ylim(0, 1.0)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# c) Final-step bar chart
final_step = eval_df['step'].max()
final_row  = eval_df[eval_df['step'] == final_step].iloc[0]
final_scores = {
    'Precision': final_row['eval_precision'],
    'Recall':    final_row['eval_recall'],
    'F1-score':  final_row['eval_f1']
}

plt.figure(figsize=(8, 5), dpi=200)
bars = plt.bar(final_scores.keys(), final_scores.values())
for bar, score in zip(bars, final_scores.values()):
    plt.text(bar.get_x() + bar.get_width()/2, score + 0.01,
             f"{score:.3f}", ha='center')
plt.ylim(0, 1.0)
plt.title(f"Final Metrics at Step {int(final_step)}")
plt.grid(axis='y')
plt.tight_layout()
plt.show()

